In [ ]:
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import os
import numpy as np
from sklearn.utils import shuffle
from random import randint
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import concatenate

In [ ]:
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    label = 0
    
    for labels in os.listdir(directory): #Main Directory where each class label is present as folder name.
        if labels == 'glacier': #Folder contain Glacier Images get the '2' class label.
            label = 2
        elif labels == 'sea':
            label = 3
        elif labels == 'buildings':
            label = 0
        elif labels == 'forest':
            label = 1

        for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(100,100)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(label)
    
    return shuffle(Images,Labels,random_state=817328462) #Shuffle the dataset you just prepared.

def get_classlabel(class_code):
    labels = {2:'glacier', 3:'sea', 0:'buildings', 1:'forest'}
    
    return labels[class_code]

In [ ]:
Images0, Labels0 = get_images('E:/fault diagnose/12-10/data whole/2000eachclass/seg_train/seg_train/0/') #Extract the training images from the folders. This folder contains all training images of buildings class.
Images0 = np.array(Images0) #converting the list of images to numpy array.
Labels0 = np.array(Labels0)

Images1, Labels1 = get_images('E:/fault diagnose/12-10/data whole/2000eachclass/seg_train/seg_train/1/') 
Images1 = np.array(Images1) #converting the list of images to numpy array.
Labels1 = np.array(Labels1)

Images2, Labels2 = get_images('E:/fault diagnose/12-10/data whole/2000eachclass/seg_train/seg_train/2/') 
Images2 = np.array(Images2) #converting the list of images to numpy array.
Labels2 = np.array(Labels2)


Images3, Labels3 = get_images('E:/fault diagnose/12-10/data whole/2000eachclass/seg_train/seg_train/4/')
Images3 = np.array(Images3) #converting the list of images to numpy array.
Labels3 = np.array(Labels3)

In [ ]:
a=0.5 # Total image number of each class is 2000. So the split ratio is 0.5 for training data
Images0_train, Images0_test, Labels0_train, Labels0_test = train_test_split(Images0, Labels0, test_size=a)
Images1_train, Images1_test, Labels1_train, Labels1_test = train_test_split(Images1, Labels1, test_size=a)
Images2_train, Images2_test, Labels2_train, Labels2_test = train_test_split(Images2, Labels2, test_size=a)
Images3_train, Images3_test, Labels3_train, Labels3_test = train_test_split(Images3, Labels3, test_size=a)

Images1_train.shape

In [ ]:
Images0=np.concatenate((Images0_train,Images1_train,Images2_train,Images3_train))
Labels0=np.concatenate((Labels0_train,Labels1_train,Labels2_train,Labels3_train))
Images,Labels=shuffle(Images0,Labels0) # shuffle the training data.
print("Shape of Images:",Images.shape)
print("Shape of Labels:",Labels.shape)

In [ ]:
from tensorflow.keras.layers import Input
input_tensor = Input(shape=(100,100, 3))
input_tensor

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
#base_model = Xception(weights='imagenet',include_top = False,input_tensor=input_tensor)
base_model = VGG16(weights='imagenet',include_top = False,input_tensor=input_tensor)
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense
from tensorflow.keras.models import Model
im = base_model.output
im = GlobalAveragePooling2D()(im)
im = Dense(512, activation='relu')(im)
predictions = Dense(4, activation='softmax')(im)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in model.layers[:20]: # the former 20 layers of the transfer learning network is frozen. 
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
trained = model.fit(Images,Labels,epochs=20,validation_split=0)

In [ ]:
test_images,test_labels = get_images('E:/fault diagnose/12-10/data whole/2000eachclass/seg_test/seg_test/4types/') # testing data directory.
test_images = np.array(test_images)
test_labels = np.array(test_labels)
model.evaluate(test_images,test_labels, verbose=1)